In [ ]:
import warnings warnings.filterwarnings("ignore") import matplotlib.pyplot as plt import numpy as np
import librosa
import IPython.display as ipd from scipy.io import wavfile
# load audio dataset from kaggle
train_audio_path = '/kaggle/working/tensorflow-speech-recognition-challenge/train/train/audio/'
# find the sample rate
sample_rate=40000
samples, sample_rate = librosa.load(train_audio_path+'yes/0a7c2a8d_nohash_0.wav', sr = sample_rate)
ipd.Audio(samples, rate=sample_rate)
sample_rate=16000
samples, sample_rate = librosa.load(train_audio_path+'yes/0a7c2a8d_nohash_0.wav', sr = sample_rate)
ipd.Audio(samples, rate=sample_rate)
# resampling
samples = librosa.resample(samples, sample_rate, 8000) ipd.Audio(samples, rate=8000) labels=os.listdir(train_audio_path)
no_of_recordings=[]
for label in labels:
waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')] no_of_recordings.append(len(waves))
#plot
plt.figure(figsize=(30,5))
index = np.arange(len(labels))
plt.bar(index, no_of_recordings) plt.xlabel('Commands', fontsize=12) plt.ylabel('No of recordings', fontsize=12) plt.xticks(index, labels, fontsize=15, rotation=60) plt.title('No. of recordings for each command') plt.show()
duration_of_recordings=[]
for label in labels:
waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]

In [ ]:
for wav in waves:
sample_rate, samples = wavfile.read(train_audio_path + '/' + label + '/' + wav) duration_of_recordings.append(float(len(samples)/sample_rate))
plt.hist(np.array(duration_of_recordings))
all_wave = []
all_label = []
for label in labels:
print(label)
waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
for wav in waves:
samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000) samples = librosa.resample(samples, sample_rate, 8000)
if(len(samples)== 8000) : all_wave.append(samples) all_label.append(label)
from sklearn.preprocessing import LabelEncoder le = LabelEncoder() y=le.fit_transform(all_label)
classes= list(le.classes_)
# split dataset
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)
# Designing a model
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()
inputs = Input(shape=(8000,1))
#First Conv1D layer
conv = Conv1D(filters=8,kernel_size=13, padding='valid', activation='relu', strides=1)(inputs) conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)
#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv) conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)
#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv) conv = MaxPooling1D(3)(conv)

In [ ]:
conv = Dropout(0.3)(conv)
#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv) conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)
#Flatten layer
conv = Flatten()(conv)
#Dense Layer 1
conv = Dense(256, activation='relu')(conv) conv = Dropout(0.3)(conv)
#Dense Layer 2
conv = Dense(128, activation='relu')(conv) conv = Dropout(0.3)(conv)
outputs = Dense(len(labels), activation='softmax')(conv)
model = Model(inputs, outputs) model.summary()
# Compile & evaluate model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) history=model.fit(x_tr, y_tr ,epochs=100, callbacks=[es,mc], batch_size=32, validation_data=(x_val,y_val))
#predict text for given audio
import speech_recognition as sr r = sr.Recognizer()
with sr.Microphone() as source:
print("Speak Anything :") audio = r.listen(source) try:
text = r.recognize_google(audio)
print("You said : {}".format(text)) except:
print("Sorry could not recognize what you said")